In [ ]:
import numpy as np
import timeit
np.random.seed(1000)

In [ ]:
 def evalf(x, n, l ):
  assert type(x) is np.ndarray
  assert len(x) == n
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = l/2*np.matmul(x.T,x) + 0.5 * (fval)**2
  return (fval)
#x = np.array([1,1])
#evalf(x, 2,1)

In [ ]:
def evalg(x, n, l):
  assert type(x) is np.ndarray
  assert len(x) == n
  grad = l*x + np.matmul(A.T, np.matmul(A, x) - y)
  return grad

In [ ]:
def evalh(x,n,l):
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments
  assert len(x) == n #do not allow arbitrary size arguments
  return np.matmul(A.T,A) + l*np.identity(n)

In [ ]:
#line search type
BACKTRACKING_LINE_SEARCH = 2

In [ ]:
def compute_D_k(x,n):
  assert type(x) is np.ndarray
  assert len(x) == n
  hess = evalh(x,n)
  return np.linalg.inv(hess)

In [ ]:
def compute_steplength_backtracking_scaled_direction(x,n,l, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function
  assert type(x) is np.ndarray and len(x) == n
  assert type(gradf) is np.ndarray and len(gradf) == n
  #assert type(direction) is np.ndarray and len(direction) == 2

  #assert type(alpha_start) is float and alpha_start>=0.
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0.


  #Complete the code
  alpha = alpha_start
  gradf = evalg(x,n,l)
  p=direction
  #np.matmul(np.matrix.transpose(gradf), p)
  while evalf(x+alpha*p,n,l) > evalf(x,n,l) + gamma*alpha*np.matmul(np.matrix.transpose(gradf), p) :
    alpha = rho*alpha

  return alpha

In [ ]:
import math
def find_minimizer_Newtonmethod(start_x, n,l, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments
  assert len(start_x) == n #do not allow arbitrary size arguments
  assert type(tol) is float and tol>=0

  x = start_x
  g_x = evalg(x,n,l)
  h_x = evalh(x,n,l)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    D_k=np.linalg.inv(evalh(x,n,l))
    direction = np.matmul(D_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x,n,l,g_x, direction, alpha_start, rho, gamma)
    else:
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1 #increment iteration
    g_x = evalg(x, n,l) #compute gradient at new point
  return x, evalf(x,n,l), k

In [ ]:

def find_minimizer_BFGS_method(start_x, n,l, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments
  assert len(start_x) == n #do not allow arbitrary size arguments
  assert type(tol) is float and tol>=0

  x = start_x
  x0 = x
  g_x = evalg(x,n,l)
  g0 = g_x


  if line_search_type == BACKTRACKING_LINE_SEARCH:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here

    x0 = x
    g_x = evalg(x,n,l)
    g0 = g_x

    if k==0:
      B_k=np.identity(n)
    else:

      I = np.identity(n)

      mu_k = 1/np.matmul(np.transpose(y_k),s_k)

      B_k = np.add(np.matmul(np.matmul(np.subtract(I, mu_k*np.outer( s_k, np.transpose(y_k))),B_k), np.subtract(I, mu_k*np.outer(y_k,np.transpose(s_k)))), mu_k*np.outer( s_k, np.transpose(s_k)))
    direction = np.matmul(B_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x,n,l,g_x, direction, alpha_start, rho, gamma)
    else:
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(B_k, g_x)))
    g_x = evalg(x, n,l)
    s_k = x-x0
    y_k=  g_x-g0
    k += 1 #increment iteration
  return x, evalf(x,n,l), k

In [ ]:
from tqdm import tqdm

In [ ]:
#Code for Newton method
my_tol= 1e-3
alpha = 0.99
rho = 0.5
gamma = 0.5
import numpy as np
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1) #random noise
#For each value of dimension in the ds array, we will check the behavior of Newton method
for i in tqdm(range(np.size(ds))):
  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  my_start_x = np.zeros((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  #call Newton method with A,y,lambda and obtain the optimal solution x_opt
  x_opt, opt_fval, num_iters = find_minimizer_Newtonmethod(my_start_x, d,lambda_reg, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
  #x_opt = Newton(A,y,lambda_reg)
  newtontime = timeit.default_timer() - start #time is in seconds
  #print the total time and the L2 norm difference || x_opt - xorig|| for Newton method
  diff = np.subtract(x_opt, xorig)
  print("\n\nd:", d)
  print("Total time:", newtontime)
  print("L2 norm difference ||Ax* - y||^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_opt), y))**2)
  print("L2 norm difference || x_opt - xorig||^2: ", np.linalg.norm(diff)**2)

 10%|████████▍                                                                           | 1/10 [00:00<00:08,  1.03it/s]



d: 1000
Total time: 0.9500885452143848
L2 norm difference ||Ax* - y||^2:  2.1594356171272403e-07
L2 norm difference || x_opt - xorig||^2:  817.7202986394225


 20%|████████████████▊                                                                   | 2/10 [00:08<00:40,  5.00s/it]



d: 5000
Total time: 7.79838355910033
L2 norm difference ||Ax* - y||^2:  5.20105640960736e-08
L2 norm difference || x_opt - xorig||^2:  4774.913359906696


 30%|█████████████████████████▏                                                          | 3/10 [01:02<03:09, 27.08s/it]



d: 10000
Total time: 53.28306541312486
L2 norm difference ||Ax* - y||^2:  1.7552731147651013e-08
L2 norm difference || x_opt - xorig||^2:  9825.861490427435


 40%|█████████████████████████████████▏                                                 | 4/10 [06:25<14:23, 143.93s/it]



d: 20000
Total time: 322.949057132937
L2 norm difference ||Ax* - y||^2:  1.1153600947149368e-08
L2 norm difference || x_opt - xorig||^2:  19778.091457899427


 40%|█████████████████████████████████▏                                                 | 4/10 [20:12<30:19, 303.19s/it]


KeyboardInterrupt: 

**In the google collab it took d = 10000 to stop and on the server it went to d = 20000**

$\textbf{Tabulation}$

In [ ]:
import pandas as pd
Dimention_of_A=[[200, 1000],[200,5000],[200,10000]]
Total_Time = [0.7243036769999947, 51.73465628299999, 445.374148636]
L2_norm_difference1= [ 2.1594356170004829e-07, 5.201056420770291e-08,  1.7552731181366058e-08]
L2_norm_difference2= [817.7202986394223, 4774.913359906627
 , 9825.861490427436]
data = {'Dimention_of_A ': Dimention_of_A,'Total Time':Total_Time,'||Ax^* - y||_2^2':L2_norm_difference1,'||x^* - x_{orig}||_2^2':L2_norm_difference2}
df = pd.DataFrame(data)
df

,Dimention_of_A,Total Time,||Ax^* - y||_2^2,||x^* - x_{orig}||_2^2
0,"[200, 1000]",0.724304,2.159436e-07,817.720299
1,"[200, 5000]",51.734656,5.201056e-08,4774.913360
2,"[200, 10000]",445.374149,1.755273e-08,9825.861490


In [ ]:
print("Using BFGS Method")
my_tol = my_tol= 1e-3
alpha = 0.99
rho = 0.5
gamma = 0.5
import numpy as np
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1) #random noise
#For each value of dimension in the ds array, we will check the behavior of BFGS method
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  my_start_x = np.zeros((d,1))
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps
  #print the total time, ||Ax_opt-y||^2 and the L2 norm difference || x_opt - xorig||^2  for Newton method
  start = timeit.default_timer()
  #call BFGS method with A,y,lambda and obtain the optimal solution x_opt_bfgs
  x_star_bfgs, opt_fval, num_iters= find_minimizer_BFGS_method(my_start_x,d,lambda_reg,my_tol,BACKTRACKING_LINE_SEARCH,0.99, 0.5,0.5)

  #x_opt_bfgs = BFGS(A,y,lambda_reg)
  bfgstime = timeit.default_timer() - start #time is in seconds
  #print the total time, ||Ax_opt_bfgs-y||^2 and the L2 norm difference || x_opt_bfgs -  xorig||^2 for BFGS method
  diff = np.subtract(x_star_bfgs, xorig)
  print("d:", d)
  print("Total time:", bfgstime)
  print("L2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_star_bfgs), y))**2)
  print("L2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)

Using BFGS Method
d: 1000
Total time: 5.30786412069574
L2 norm difference ||Ax* - y||_2^2:  2.1792993993142904e-07
L2 norm difference || x_opt - xorig||:  817.7202987075375
d: 5000
Total time: 393.4668908366002
L2 norm difference ||Ax* - y||_2^2:  4.8410708212598343e-08
L2 norm difference || x_opt - xorig||:  4774.913359904622


KeyboardInterrupt: 

**till d = 10000 the code took more than 30 minutes to run**

$\textbf{Tabulation}$

In [ ]:
import pandas as pd
Dimention_of_A=[[200, 1000],[200,5000],[200,10000]]
Total_Time = [19.19767696100007, 1388.179293315,10504.021474666]
L2_norm_difference1= [2.1792993971933117e-07, 4.8410711664385876e-08,1.805632077533474e-08]
L2_norm_difference2= [817.7202987075317, 4774.913359904525,9825.861490430612]
data = {'Dimention_of_A ': Dimention_of_A,'Total Time':Total_Time,'||Ax^* - y||_2^2':L2_norm_difference1,'||x^* - x_{orig}||_2^2':L2_norm_difference2}
df = pd.DataFrame(data)
df

,Dimention_of_A,Total Time,||Ax^* - y||_2^2,||x^* - x_{orig}||_2^2
0,"[200, 1000]",19.197677,2.179299e-07,817.720299
1,"[200, 5000]",1388.179293,4.841071e-08,4774.913360
2,"[200, 10000]",10504.021475,1.805632e-08,9825.861490


$\textbf{Report the dimension where failure occurs respectively for Newton and BFGS method.}$

The failure occures for newton's method after size of A = (200, 10000) i.e. after d = 10000

The failure occures for BFGS method after size of A = (200, 10000) i.e. after  d =10000